# Визуализация целевой переменной в двухмерном пространстве

## PCA / LDA / NCA

Этот notebook демонстрирует три метода снижения размерности для визуализации целевой переменной:

1. **PCA (Principal Component Analysis)** - максимизирует дисперсию
2. **LDA (Linear Discriminant Analysis)** - максимизирует разделимость классов
3. **NCA (Neighborhood Components Analysis)** - оптимизирует kNN классификацию

In [ ]:
# Импорты
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import NeighborhoodComponentsAnalysis as NCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Настройки визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (20, 6)
plt.rcParams['font.size'] = 11

RANDOM_STATE = 42
SAMPLE_SIZE = 10000  # Для ускорения (особенно NCA)

print("✓ Библиотеки загружены")

## 1. Загрузка данных

In [ ]:
# Путь к данным
DATA_DIR = Path("data")
TRAIN_FILE = DATA_DIR / "churn_train_ul.parquet"

print(f"Поиск файла: {TRAIN_FILE}")

# Проверка существования
if TRAIN_FILE.exists():
    print(f"✓ Файл найден! Загрузка...")
    df = pd.read_parquet(TRAIN_FILE)
    print(f"✓ Загружено: {df.shape}")
    print(f"\nПервые строки:")
    display(df.head())
    
else:
    print(f"⚠ Файл не найден: {TRAIN_FILE}")
    print("\nСоздание СИНТЕТИЧЕСКИХ данных для демонстрации...")
    
    # Синтетические данные
    np.random.seed(RANDOM_STATE)
    n_samples = 5000
    n_features = 50
    
    # Класс 0 (не churn)
    X_class0 = np.random.randn(int(n_samples * 0.985), n_features) * 0.8
    y_class0 = np.zeros(int(n_samples * 0.985))
    
    # Класс 1 (churn)
    X_class1 = np.random.randn(int(n_samples * 0.015), n_features) * 1.2 + 2
    y_class1 = np.ones(int(n_samples * 0.015))
    
    # Объединение
    X = np.vstack([X_class0, X_class1])
    y = np.hstack([y_class0, y_class1])
    
    df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(n_features)])
    df['target_churn_3m'] = y.astype(int)
    
    print(f"✓ Создано {len(df):,} синтетических записей")
    print(f"✓ Признаков: {n_features}")

print(f"\nИтоговая размерность: {df.shape}")

## 2. Подготовка данных

In [ ]:
# Целевая переменная и ID колонки
TARGET_COL = 'target_churn_3m'
ID_COLS = ['cli_code', 'client_id', 'observation_point']

# Выбор признаков
feature_cols = [col for col in df.columns if col not in ID_COLS + [TARGET_COL]]
numeric_cols = df[feature_cols].select_dtypes(include=[np.number]).columns.tolist()

print(f"Всего признаков: {len(feature_cols)}")
print(f"Числовых признаков: {len(numeric_cols)}")

# Создание X и y
X = df[numeric_cols].copy()
y = df[TARGET_COL].copy()

# Обработка пропусков
if X.isnull().any().any():
    print("Заполнение пропусков медианой...")
    X = X.fillna(X.median())

# Удаление константных признаков
constant_cols = [col for col in X.columns if X[col].nunique() <= 1]
if constant_cols:
    print(f"Удаление {len(constant_cols)} константных признаков...")
    X = X.drop(columns=constant_cols)

print(f"\nФинальная размерность: {X.shape}")
print(f"\nРаспределение целевой переменной:")
print(y.value_counts())
print(f"\nChurn rate: {y.mean():.4f} ({y.mean()*100:.2f}%)")

In [ ]:
# Сэмплирование для ускорения (если нужно)
if len(X) > SAMPLE_SIZE:
    print(f"Сэмплирование {SAMPLE_SIZE:,} записей для визуализации...")
    X_sample, _, y_sample, _ = train_test_split(
        X, y, 
        train_size=SAMPLE_SIZE, 
        stratify=y, 
        random_state=RANDOM_STATE
    )
    X = X_sample
    y = y_sample
    print(f"✓ Выборка: {X.shape}")

# Стандартизация (критично важно!)
print("\nСтандартизация признаков...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("✓ Данные стандартизированы")

## 3. Применение методов снижения размерности

### 3.1 PCA (Principal Component Analysis)

In [ ]:
print("[PCA] Principal Component Analysis...")
pca = PCA(n_components=2, random_state=RANDOM_STATE)
X_pca = pca.fit_transform(X_scaled)

explained_var_pca = pca.explained_variance_ratio_
print(f"✓ Explained variance:")
print(f"  PC1: {explained_var_pca[0]:.4f} ({explained_var_pca[0]*100:.2f}%)")
print(f"  PC2: {explained_var_pca[1]:.4f} ({explained_var_pca[1]*100:.2f}%)")
print(f"  Total: {explained_var_pca.sum():.4f} ({explained_var_pca.sum()*100:.2f}%)")

### 3.2 LDA (Linear Discriminant Analysis)

In [ ]:
print("[LDA] Linear Discriminant Analysis...")

# LDA для бинарной классификации дает 1 компоненту
# Добавляем вторую через PCA для визуализации
lda = LDA(n_components=1)
X_lda_comp1 = lda.fit_transform(X_scaled, y)

# Вторая компонента - PCA
pca_for_lda = PCA(n_components=2)
X_pca_temp = pca_for_lda.fit_transform(X_scaled)
X_lda = np.column_stack([X_lda_comp1, X_pca_temp[:, 1]])

print("✓ LDA применен (1D discriminant + 1D PCA)")

### 3.3 NCA (Neighborhood Components Analysis)

In [ ]:
print("[NCA] Neighborhood Components Analysis...")
print("⚠ NCA может занять несколько минут на больших данных...")

# NCA очень медленный - ограничим размер выборки
nca = NCA(n_components=2, random_state=RANDOM_STATE, max_iter=100, verbose=0)

if len(X_scaled) > 5000:
    print(f"Используем {5000} образцов для NCA...")
    idx_nca = np.random.choice(len(X_scaled), 5000, replace=False)
    X_nca = nca.fit_transform(X_scaled[idx_nca], y.iloc[idx_nca])
    y_nca = y.iloc[idx_nca]
else:
    X_nca = nca.fit_transform(X_scaled, y)
    y_nca = y

print("✓ NCA завершен")

## 4. Визуализация

In [ ]:
# Создание фигуры
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
fig.suptitle('Визуализация целевой переменной в двухмерном пространстве', 
             fontsize=16, fontweight='bold', y=1.02)

# Цвета
colors = ['#2ecc71', '#e74c3c']  # Зеленый для No Churn, Красный для Churn
labels = ['No Churn (0)', 'Churn (1)']

# --- PCA ---
ax = axes[0]
for class_val in [0, 1]:
    mask = y == class_val
    ax.scatter(X_pca[mask, 0], X_pca[mask, 1],
              c=colors[class_val],
              label=labels[class_val],
              alpha=0.6,
              s=20,
              edgecolors='black',
              linewidth=0.3)

ax.set_xlabel(f'PC1 ({explained_var_pca[0]:.2%} variance)', fontsize=11)
ax.set_ylabel(f'PC2 ({explained_var_pca[1]:.2%} variance)', fontsize=11)
ax.set_title('PCA\n(Principal Component Analysis)', fontsize=13, fontweight='bold')
ax.legend(loc='best', framealpha=0.9)
ax.grid(True, alpha=0.3)

# --- LDA ---
ax = axes[1]
for class_val in [0, 1]:
    mask = y == class_val
    ax.scatter(X_lda[mask, 0], X_lda[mask, 1],
              c=colors[class_val],
              label=labels[class_val],
              alpha=0.6,
              s=20,
              edgecolors='black',
              linewidth=0.3)

ax.set_xlabel('LD1 (Linear Discriminant)', fontsize=11)
ax.set_ylabel('PC1 (PCA Component)', fontsize=11)
ax.set_title('LDA\n(Linear Discriminant Analysis)', fontsize=13, fontweight='bold')
ax.legend(loc='best', framealpha=0.9)
ax.grid(True, alpha=0.3)

# --- NCA ---
ax = axes[2]
for class_val in [0, 1]:
    mask = y_nca == class_val
    ax.scatter(X_nca[mask, 0], X_nca[mask, 1],
              c=colors[class_val],
              label=labels[class_val],
              alpha=0.6,
              s=20,
              edgecolors='black',
              linewidth=0.3)

ax.set_xlabel('NCA Component 1', fontsize=11)
ax.set_ylabel('NCA Component 2', fontsize=11)
ax.set_title('NCA\n(Neighborhood Components Analysis)', fontsize=13, fontweight='bold')
ax.legend(loc='best', framealpha=0.9)
ax.grid(True, alpha=0.3)

plt.tight_layout()

# Сохранение
output_dir = Path("figures")
output_dir.mkdir(exist_ok=True)
output_file = output_dir / "target_visualization_pca_lda_nca.png"
plt.savefig(output_file, dpi=300, bbox_inches='tight')
print(f"\n✓ Визуализация сохранена: {output_file}")

plt.show()

## 5. Интерпретация результатов

In [ ]:
print("="*80)
print("ИНТЕРПРЕТАЦИЯ РЕЗУЛЬТАТОВ")
print("="*80)

print("\n[PCA] Principal Component Analysis:")
print("  • Несупервизированный метод (не использует метки классов)")
print("  • Находит направления максимальной дисперсии")
print(f"  • Первые 2 компоненты объясняют {explained_var_pca.sum():.2%} дисперсии")
print("  • Полезно для понимания общей структуры данных")

print("\n[LDA] Linear Discriminant Analysis:")
print("  • Супервизированный метод (использует метки классов)")
print("  • Максимизирует разделимость между классами")
print("  • Для бинарной классификации: 1 дискриминант + 1 PCA компонента")
print("  • Лучше для визуализации разделимости классов")

print("\n[NCA] Neighborhood Components Analysis:")
print("  • Супервизированный метод (использует метки классов)")
print("  • Оптимизирует метрику для kNN классификации")
print("  • Учитывает локальную структуру данных")
print("  • Наиболее вычислительно затратный метод")

print("\n" + "="*80)
print("РЕКОМЕНДАЦИИ:")
print("="*80)
print("1. PCA - для исследования общей структуры данных")
print("2. LDA - для максимальной разделимости классов")
print("3. NCA - для учета локальных паттернов и улучшения kNN")
print("\n✓ Анализ завершен!")
print("="*80)

## 6. Дополнительный анализ: Отдельные графики

In [ ]:
# Более детальный PCA график
fig, ax = plt.subplots(figsize=(10, 8))

for class_val in [0, 1]:
    mask = y == class_val
    ax.scatter(X_pca[mask, 0], X_pca[mask, 1],
              c=colors[class_val],
              label=labels[class_val],
              alpha=0.5,
              s=30,
              edgecolors='black',
              linewidth=0.5)

ax.set_xlabel(f'PC1 ({explained_var_pca[0]:.2%} variance)', fontsize=12)
ax.set_ylabel(f'PC2 ({explained_var_pca[1]:.2%} variance)', fontsize=12)
ax.set_title('PCA - Детальная визуализация', fontsize=14, fontweight='bold')
ax.legend(loc='best', framealpha=0.9, fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / "pca_detailed.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Более детальный LDA график
fig, ax = plt.subplots(figsize=(10, 8))

for class_val in [0, 1]:
    mask = y == class_val
    ax.scatter(X_lda[mask, 0], X_lda[mask, 1],
              c=colors[class_val],
              label=labels[class_val],
              alpha=0.5,
              s=30,
              edgecolors='black',
              linewidth=0.5)

ax.set_xlabel('LD1 (Linear Discriminant)', fontsize=12)
ax.set_ylabel('PC1 (PCA Component)', fontsize=12)
ax.set_title('LDA - Детальная визуализация', fontsize=14, fontweight='bold')
ax.legend(loc='best', framealpha=0.9, fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / "lda_detailed.png", dpi=300, bbox_inches='tight')
plt.show()

---

## Итоги

Мы успешно визуализировали целевую переменную в двухмерном пространстве тремя методами:

- **PCA**: Показывает общую структуру данных
- **LDA**: Максимизирует разделимость классов
- **NCA**: Оптимизирует для kNN классификации

Все визуализации сохранены в директории `figures/`